# Python で気軽に化学・化学工学
# 第 8 章 モデル y = f(x) を構築して、新たなサンプルの y を推定する
## 8.6 さまざまな解析の自動化・効率化

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

わからないことがありましたら、関係する単語やエラーの文章などでウェブ検索してご自身で調べてみましょう。

## for文によるハイパーパラメータの決定

### for 文
- 指定した回数だけ処理を繰り返す場合に使用
- 字下げ (インデント) を揃えることで、for 文で繰り返しをするコードを指定
- 字下げしないと for 文の中とは認識されず、繰り返されないため注意
- 半角スペース 4 つもしくは <kbd>Tab</kbd> が字下げの決まり
- <kbd>Delete</kbd> や <kbd>Back space</kbd> や <kbd>Shift</kbd>+<kbd>Tab</kbd> で字下げを削除

In [ ]:
for i in range(10):
    print(i)

In [ ]:
sum_of_number = 0
for number in range(1, 11):
    sum_of_number = sum_of_number + number
    print(number, sum_of_number)
print(sum_of_number)

In [ ]:
sum_of_number = 0
for number in range(5, 30, 3):
    sum_of_number += number # sum_of_number = sum_of_number + number と同じ
    print(number, sum_of_number)
print(sum_of_number)

In [ ]:
numbers = [] # 空の list を作成
for number in range(30, 21, -2):
    numbers.append(number)
    print(numbers)

list の要素を for 文で順番に選択する

In [ ]:
words = ['windows', 'mac', 'linax']
for word in words:
    print(word, len(word))

In [ ]:
# index番号も欲しいとき、enumerate()を使います
for index, word in enumerate(words):
    print(index, word, len(word))

### クロスバリデーションによる PLS の主成分数の決定

前回と同様に沸点のデータセットを扱います。

### 沸点のデータセット(descriptors_8_with_boiling_point.csv)
Hall and Story が収集した[沸点のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci960375x)。294 個の化合物について、沸点 (Boiling Point) が測定されており、8 つの変数 (記述子) で化学構造が数値化されています。記述子は、分子量 (MolWt)、水素原子以外の原子で計算された分子量 (HeavyAtomMolWt)、価電子の数 (NumValenceElectrons)、水素原子以外の原子の数 (HeavyAtomCount)、窒素原子と酸素原子の数 (NOCount)、水素原子と炭素原子以外の原子の数 (NumHeteroatoms)、回転可能な結合の数 (NumRotatableBonds)、環の数 (RingCount) です。

データセットの読み込み

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('descriptors_8_with_boiling_point.csv', index_col=0) # 沸点のデータセットの読み込み

In [ ]:
dataset.shape

In [ ]:
x = dataset.iloc[:, 1:] # 記述子を説明変数 x とします

In [ ]:
y = dataset.iloc[:, 0] # 沸点を目的変数 y とします

データセットのトレーニングデータとテストデータへの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=94, shuffle=True, random_state=99)

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの説明変数の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

In [ ]:
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std() # トレーニングデータの目的変数の標準化

PLS

In [ ]:
from sklearn import model_selection #クロスバリデーションにより推定値を計算するために使用
from sklearn.cross_decomposition import PLSRegression # PLS モデル構築やモデルを用いた y の値の推定に使用
from sklearn import metrics # r2 等の指標の計算に使用

In [ ]:
max_number_of_components = 8 # 最大の主成分数

In [ ]:
fold_number = 5 # クロスバリデーションのfold数

In [ ]:
components = [] # 空の list を作成して、主成分数をこの変数に追加していきます
r2_in_cv_all = [] # 空の list を作成して、主成分数ごとのクロスバリデーション後の r2 をこの list に追加していきます

主成分数を 1, 2, ..., max_number_of_components-1, max_number_of_components と変えて、それぞれクロスバリデーションにより推定値を計算

In [ ]:
for component in range(1, max_number_of_components+1):
    model = PLSRegression(n_components=component) # PLS モデルの宣言
    estimated_y_in_cv = pd.DataFrame(model_selection.cross_val_predict(model, autoscaled_x_train, autoscaled_y_train, cv=fold_number)) # クロスバリデーション推定値の計算し、DataFrame型に変換
    estimated_y_in_cv = estimated_y_in_cv * y_train.std() + y_train.mean() # スケールをもとに戻す
    r2_in_cv = metrics.r2_score(y_train, estimated_y_in_cv) # r2 を計算
    print(component, r2_in_cv) # 主成分数と r2 を表示
    r2_in_cv_all.append(r2_in_cv)  # r2 を追加
    components.append(component) # 主成分数を追加

In [ ]:
r2_in_cv_all # 念のため確認

主成分数ごとのクロスバリデーション後の r<sup>2</sup> を図で確認

In [ ]:
import matplotlib.pyplot as plt # 図の描画に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(components, r2_in_cv_all) # 散布図
plt.xlabel('number of components') # x 軸の名前
plt.ylabel('cross-validated r2') # y 軸の名前
plt.show() # 以上の設定で描画

In [ ]:
max(r2_in_cv_all) # クロスバリデーション後の r2 の最大値

In [ ]:
r2_in_cv_all.index(max(r2_in_cv_all)) # クロスバリデーション後の r2 が最大値となる index 番号

In [ ]:
optimal_component_number = components[r2_in_cv_all.index(max(r2_in_cv_all))] # 最良の主成分数を optimal_component_number に代入

In [ ]:
optimal_component_number # 念のため確認

最良の主成分数で PLS モデル構築

In [ ]:
model = PLSRegression(n_components=optimal_component_number)

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # 回帰モデルの構築

標準回帰係数

In [ ]:
standard_regression_coefficients = pd.DataFrame(model.coef_) # 標準回帰係数を Pandas の DataFrame 型に変換

In [ ]:
standard_regression_coefficients.index = x_train.columns # 変数に対応する名前を、元のデータの変数名に

In [ ]:
standard_regression_coefficients.columns = ['standard_regression_coefficients'] # 列名を変更

In [ ]:
standard_regression_coefficients.to_csv('standard_regression_coefficients.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # トレーニングデータの y の値を推定し、Pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # テストデータの y の値を推定して Pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

### 練習問題
k-NN による回帰分析において、k の値を 1, 2, ..., 10 と変えてそれぞれクロスバリデーションを行い、r<sup>2</sup> が最大となる k の値を選択しましょう。コード例はサンプルNotebookの一番下にあります。

### 練習問題
k-NN によるクラス分類において、k の値を 1, 2, ..., 10 と変えてそれぞれクロスバリデーションを行い、正解率が最大となる k の値を選択しましょう。あやめのデータを用いてください。コード例はサンプルNotebookの一番下にあります。

## f 文による複数の手法を用いたデータ解析

### if 文
- 条件によって処理を変化させる場合に使用
- 字下げ (インデント) を揃えることで、if文の各条件に対応するコードを指定
- 字下げしないと条件の中の処理とは認識されないため注意
- 半角スペース 4 つもしくは <kbd>Tab</kbd> が字下げの決まり
- <kbd>Delete</kbd> や <kbd>Back space</kbd> や <kbd>Shift</kbd>+<kbd>Tab</kbd> で字下げを削除

In [ ]:
number = 15

In [ ]:
if number > 10:
    print(number, 'is greater than 10')

In [ ]:
if number > 10:
    print(number, 'is greater than 10')
else:
    print(number, 'is less than 10')

In [ ]:
if number == 10:
    print(number, 'is equal to 10')
elif number > 10:
    print(number, 'is greater than 10')
else:
    print(number, 'is less than 10')

条件式に文字列を用いることもできます。

In [ ]:
alcohol_drink = 'beer'

In [ ]:
if alcohol_drink == 'beer':
    ethanol_conc = 'about 5 %'
elif alcohol_drink == 'sake':
    ethanol_conc = 'about 15 %'
elif alcohol_drink == 'wine':
    ethanol_conc = 'about 14 %'
elif alcohol_drink == 'whiskey' or alcohol_drink == 'whisky': # 2 つの条件式のうちどちらかを満たせばその後の処理を実行
    ethanol_conc = 'more than 40 %'
else:
    ethanol_conc = 'unknown'
print('Ethanol concentration in', alcohol_drink, ':', ethanol_conc)

OLS 法, PLS 法で回帰分析

データセットの読み込み

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('descriptors_8_with_boiling_point.csv', index_col=0) # 沸点のデータの読み込み

In [ ]:
x = dataset.iloc[:, 1:] # 記述子を説明変数 x とします

In [ ]:
y = dataset.iloc[:, 0] # 沸点を目的変数 y とします

データセットのトレーニングデータとテストデータへの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=94, shuffle=True, random_state=99)

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの説明変数の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

In [ ]:
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std() # トレーニングデータの目的変数の標準化

In [ ]:
from sklearn.cross_decomposition import PLSRegression # PLS モデル構築やモデルを用いた y の値の推定に使用
from sklearn.linear_model import LinearRegression # OLS モデル構築やモデルを用いた y の値の推定に使用
from sklearn import metrics # r2 等の指標の計算に使用

OLS 法か PLS 法を選択して回帰分析

In [ ]:
regression_method = 'OLS' # 'OLS' or 'PLS'
number_of_components = 2 # PLS における主成分数

In [ ]:
if regression_method == 'OLS':
    model = LinearRegression()
elif regression_method == 'PLS':
    model = PLSRegression(n_components=number_of_components)
else:
    print('The regression method is not prepared. Please check the variable of `regression_method`.')

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # 回帰モデルの構築

標準回帰係数

In [ ]:
standard_regression_coefficients = pd.DataFrame(model.coef_) # Pandas の DataFrame 型に変換

In [ ]:
standard_regression_coefficients.index = x_train.columns # 変数に対応する名前を、元のデータの変数名に

In [ ]:
standard_regression_coefficients.columns = ['standard_regression_coefficients'] # 列名を変更

In [ ]:
standard_regression_coefficients.to_csv('standard_regression_coefficients.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # Pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # Pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

`regression_method` に代入するセルに戻り、'OLS' を 'PLS' に変更して再度実行しましょう。 

### 練習問題
OLS, PLS に加えて k-NN でも同様な解析ができるようなコードを作成しましょう。その際、PLS の場合は主成分数を 1, 2, 3, ..., 8 と変えて、k-NN の場合は k の値を 1, 2, ..., 10 と変えて、それぞれクロスバリデーションを行い、r<sup>2</sup> が最大となる成分数や k の値を選択してからモデル構築するようにしてください。また k-NN の場合、標準回帰係数は計算されませんので注意しましょう。コード例はサンプルNotebookの一番下にあります。

自分のデータセットをお持ちの方は、そのデータセットでも今回の内容を確認してみましょう。

### コード例
k-NN による回帰分析において、k の値を 1, 2, ..., 10 と変えてそれぞれクロスバリデーションを行い、r<sup>2</sup> が最大となる k の値を選択しましょう。

データセットの読み込み

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('descriptors_8_with_boiling_point.csv', index_col=0) # 沸点のデータの読み込み

In [ ]:
x = dataset.iloc[:, 1:] # 記述子を説明変数 x とします

In [ ]:
y = dataset.iloc[:, 0] # 沸点を目的変数 y とします

データセットのトレーニングデータとテストデータへの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=94, shuffle=True, random_state=99)

変数の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの説明変数の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

In [ ]:
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std() # トレーニングデータの目的変数の標準化

クロスバリデーションによる k-NN における k の最適化

In [ ]:
from sklearn.neighbors import KNeighborsRegressor # k-NN で使用
from sklearn import model_selection #クロスバリデーションにより推定値を計算するために使用
from sklearn import metrics # r2 等の指標の計算に使用

In [ ]:
max_number_of_k = 10 # 最大の k の数

In [ ]:
fold_number = 5 # クロスバリデーションのfold数

k を 1, 2, ..., max_number_of_k-1, max_number_of_k と変えて、それぞれクロスバリデーションにより推定値を計算

In [ ]:
r2_in_cv_all = [] # 空の list の変数を作成して、成分数ごとのクロスバリデーション後の r2 をこの変数に追加していきます
ks = [] # 同じく k の値をこの変数に追加していきます
for k in range(1, max_number_of_k+1):
    model = KNeighborsRegressor(n_neighbors=k, metric='euclidean') # k-NN モデルの宣言
    estimated_y_in_cv = pd.DataFrame(model_selection.cross_val_predict(model, autoscaled_x_train, autoscaled_y_train, cv=fold_number)) # クロスバリデーション推定値の計算し、DataFrame型に変換
    estimated_y_in_cv = estimated_y_in_cv * y_train.std() + y_train.mean() # スケールをもとに戻す
    r2_in_cv = metrics.r2_score(y_train, estimated_y_in_cv) # r2 を計算
    print(k, r2_in_cv) # k の値と r2 を表示
    r2_in_cv_all.append(r2_in_cv)  # r2 を追加
    ks.append(k) # k の値を追加

In [ ]:
r2_in_cv_all # 念のため確認

k の値ごとのクロスバリデーション後の r<sup>2</sup> を図で確認

In [ ]:
import matplotlib.pyplot as plt # 図の描画に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(ks, r2_in_cv_all) # 散布図
plt.xlabel('k') # x 軸の名前
plt.ylabel('cross-validated r2') # y 軸の名前
plt.show() # 以上の設定で描画

In [ ]:
max(r2_in_cv_all) # クロスバリデーション後の r2 の最大値

In [ ]:
r2_in_cv_all.index(max(r2_in_cv_all)) # クロスバリデーション後の r2 が最大値となる index 番号

In [ ]:
optimal_k = ks[r2_in_cv_all.index(max(r2_in_cv_all))] # 最適成分数を optimal_k に代入

In [ ]:
optimal_k # 念のため確認

k-NN

In [ ]:
model = KNeighborsRegressor(n_neighbors=optimal_k, metric='euclidean')

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # 回帰モデルの構築。k-NN では、トレーニングデータの数値データとクラスを model に格納することに対応します

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # トレーニングデータの y の値を推定し、Pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # テストデータの y の値を推定し、Pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

### コード例
k-NN によるクラス分類において、k の値を 1, 2, ..., 10 と変えてそれぞれクロスバリデーションを行い、正解率が最大となる k の値を選択しましょう。あやめのデータを用いてください。

あやめのデータの読み込み

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('iris_with_species.csv', index_col=0) # あやめのデータの読み込み

In [ ]:
x = dataset.iloc[:, 1:] # 数値データの変数を 説明変数 x とします

In [ ]:
y = dataset.iloc[:, 0] # あやめの種類を目的変数 y とします

トレーニングデータとテストデータの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=50, shuffle=True, random_state=3)

変数の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの変数の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

In [ ]:
autoscaled_x_train.mean()

クロスバリデーションによる k-NN における k の最適化

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # k-NN で使用
from sklearn import model_selection #クロスバリデーションにより推定値を計算するために使用
from sklearn import metrics # r2 等の指標の計算に使用

In [ ]:
max_number_of_k = 10 # 最大の k の数

In [ ]:
fold_number = 5 # クロスバリデーションのfold数

k を 1, 2, ..., max_number_of_k-1, max_number_of_k と変えて、それぞれクロスバリデーションにより推定

In [ ]:
accuracy_in_cv_all = [] # 空の list の変数を作成して、成分数ごとのクロスバリデーション後の 正解率 をこの変数に追加していきます
ks = [] # 同じく k の値をこの変数に追加していきます
for k in range(1, max_number_of_k+1):
    model = KNeighborsClassifier(n_neighbors=k, metric='euclidean') # k-NN モデルの宣言
    estimated_y_in_cv = pd.DataFrame(model_selection.cross_val_predict(model, autoscaled_x_train, y_train, cv=fold_number)) # クロスバリデーション推定値の計算し、DataFrame型に変換
    accuracy_in_cv = metrics.accuracy_score(y_train, estimated_y_in_cv) # 正解率を計算
    print(k, accuracy_in_cv) # k の値と r2 を表示
    accuracy_in_cv_all.append(accuracy_in_cv)  # r2 を追加
    ks.append(k) # k の値を追加

In [ ]:
accuracy_in_cv_all # 念のため確認

k の値ごとのクロスバリデーション後の正解率を図で確認

In [ ]:
import matplotlib.pyplot as plt # 図の描画に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(ks, accuracy_in_cv_all) # 散布図
plt.xlabel('k') # x 軸の名前
plt.ylabel('cross-validated accuracy') # y 軸の名前
plt.show() # 以上の設定で描画

In [ ]:
max(accuracy_in_cv_all) # クロスバリデーション後の r2 の最大値

In [ ]:
accuracy_in_cv_all.index(max(accuracy_in_cv_all)) # クロスバリデーション後の正解率が最大値となる index 番号 (同じ値が複数あるときは、もっとも index 番号の小さい番号のみ出力されます)

In [ ]:
optimal_k = ks[accuracy_in_cv_all.index(max(accuracy_in_cv_all))] # 最適な k の値を optimal_k に代入

In [ ]:
optimal_k # 念のため確認

In [ ]:
model = KNeighborsClassifier(n_neighbors=optimal_k, metric='euclidean')

In [ ]:
model.fit(autoscaled_x_train, y_train) # クラス分類モデルの構築。k-NN では、トレーニングデータの数値データとクラスを model に格納することに対応します

トレーニングデータのクラスの推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # Pandas の DataFrame 型に変換

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_class'] # 列名を変更

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの混同行列

In [ ]:
from sklearn import metrics

In [ ]:
set(y_train) # クラスの種類

In [ ]:
class_types = list(set(y_train)) # リスト型に変換。これで混同行列における縦と横のクラスの順番を定めます

In [ ]:
class_types.sort() # アルファベット順に並び替え

In [ ]:
class_types # 念のため確認

In [ ]:
metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types) # 混同行列。array 型で与えられます

In [ ]:
confusion_matrix_train = pd.DataFrame(metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types)) # Pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_train.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_train.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_train # 念のため確認

In [ ]:
confusion_matrix_train.to_csv('confusion_matrix_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_train, estimated_y_train) # 正解率

テストデータのクラスの推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
model.predict(autoscaled_x_test) # テストデータのクラスを推定。array 型で出力されます

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # Pandas の DataFrame 型に変換

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_class'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの混同行列

In [ ]:
metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types) # 混同行列。array 型で与えられます

In [ ]:
confusion_matrix_test = pd.DataFrame(metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types)) # Pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_test.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_test.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_test # 念のため確認

In [ ]:
confusion_matrix_test.to_csv('confusion_matrix_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_test, estimated_y_test) # 正解率

### コード例
OLS, PLS に加えて、k-NN でも同様な解析ができるようなコードを作成しましょう。その際、PLS の場合は成分数を 1, 2, 3, ... と変えて、k-NN の場合は k の値を 1, 2, ..., 10 と変えて、それぞれクロスバリデーションを行い、r<sup>2</sup> が最大となる成分数や k の値を選択してからモデル構築するようにしてください。また k-NN の場合は標準回帰係数はありませんので注意しましょう。

データセットの読み込み

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('descriptors_8_with_boiling_point.csv', index_col=0) # 沸点のデータの読み込み

In [ ]:
x = dataset.iloc[:, 1:] # 記述子を説明変数 x とします

In [ ]:
y = dataset.iloc[:, 0] # 沸点を目的変数 y とします

データセットのトレーニングデータとテストデータへの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=94, shuffle=True, random_state=99)

変数の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの説明変数の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

In [ ]:
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std() # トレーニングデータの目的変数の標準化

In [ ]:
from sklearn.neighbors import KNeighborsRegressor # k-NN で使用
from sklearn.cross_decomposition import PLSRegression # PLS モデル構築やモデルを用いた y の値の推定に使用
from sklearn.linear_model import LinearRegression # OLS モデル構築やモデルを用いた y の値の推定に使用
from sklearn import metrics # r2 等の指標の計算に使用

In [ ]:
regression_method = 'k-NN' # 'k-NN' or 'OLS' or 'PLS'

In [ ]:
max_number_of_k = 10 # k-NN における最大の k の数
distance_in_knn = 'euclidean' # k-NN における距離の指標
max_number_of_components = 8 # PLS における最大の主成分数
fold_number = 5 # クロスバリデーションのfold数

In [ ]:
if regression_method == 'k-NN':
    r2_in_cv_all = [] # 空の list を作成して、成分数ごとのクロスバリデーション後の r2 をこの変数に追加していきます
    ks = [] # 同じく k の値をこの list に追加していきます
    for k in range(1, max_number_of_k+1):
        model = KNeighborsRegressor(n_neighbors=k, metric='euclidean') # k-NN モデルの宣言
        estimated_y_in_cv = pd.DataFrame(model_selection.cross_val_predict(model, autoscaled_x_train, autoscaled_y_train, cv=fold_number)) # クロスバリデーション推定値の計算し、DataFrame型に変換
        estimated_y_in_cv = estimated_y_in_cv * y_train.std() + y_train.mean() # スケールをもとに戻す
        r2_in_cv = metrics.r2_score(y_train, estimated_y_in_cv) # r2 を計算
        print(k, r2_in_cv) # k の値と r2 を表示
        r2_in_cv_all.append(r2_in_cv)  # r2 を追加
        ks.append(k) # k の値を追加
    # k の値ごとのクロスバリデーション後の r2 を図で確認
    plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
    plt.scatter(ks, r2_in_cv_all) # 散布図
    plt.xlabel('k') # x 軸の名前
    plt.ylabel('cross-validated r2') # y 軸の名前
    plt.show() # 以上の設定で描画
    optimal_k = ks[r2_in_cv_all.index(max(r2_in_cv_all))] # 最良の主成分数を optimal_k に代入
    model = KNeighborsRegressor(n_neighbors=optimal_k, metric=distance_in_knn)
elif regression_method == 'OLS':
    model = LinearRegression()
elif regression_method == 'PLS':
    components = [] # 空の list を作成して、主成分数をこの変数に追加していきます
    r2_in_cv_all = [] # 空の list を作成して、成分数ごとのクロスバリデーション後の r2 をこの変数に追加していきます
    for component in range(1, max_number_of_components+1):
        model = PLSRegression(n_components=component) # PLS モデルの宣言
        estimated_y_in_cv = pd.DataFrame(model_selection.cross_val_predict(model, autoscaled_x_train, autoscaled_y_train, cv=fold_number)) # クロスバリデーション推定値の計算し、DataFrame型に変換
        estimated_y_in_cv = estimated_y_in_cv * y_train.std() + y_train.mean() # スケールをもとに戻す
        r2_in_cv = metrics.r2_score(y_train, estimated_y_in_cv) # r2 を計算
        print(component, r2_in_cv) # 主成分数と r2 を表示
        r2_in_cv_all.append(r2_in_cv)  # r2 を追加
        components.append(component) # 主成分数を追加
    # 主成分数ごとのクロスバリデーション後の r2 を図で確認
    plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
    plt.scatter(components, r2_in_cv_all) # 散布図
    plt.xlabel('number of components') # x 軸の名前
    plt.ylabel('cross-validated r2') # y 軸の名前
    plt.show() # 以上の設定で描画
    optimal_component_number = components[r2_in_cv_all.index(max(r2_in_cv_all))] # 最良の主成分数を optimal_component_number に代入
    model = PLSRegression(n_components=optimal_component_number)
else:
    print('The regression method is not prepared. Please check the variable of `regression_method`.')

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # 回帰モデルの構築

In [ ]:
# OLS と PLS の場合のみ、標準回帰係数を保存
if regression_method == 'OLS' or regression_method == 'PLS':
    standard_regression_coefficients = pd.DataFrame(model.coef_) # Pandas の DataFrame 型に変換
    standard_regression_coefficients.index = x_train.columns # 変数に対応する名前を、元のデータの変数名に
    standard_regression_coefficients.columns = ['standard_regression_coefficients'] # 列名を変更
    standard_regression_coefficients.to_csv('standard_regression_coefficients.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # Pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # Pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE